week3 dashboard application

In [ ]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                dcc.Dropdown(id='site-dropdown',
                                                options=[
                                                    {'label': 'All Sites', 'value': 'ALL'},
                                                    {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                    {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                                    {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                    {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                                ],
                                                value='ALL',
                                                searchable=True,
                                                placeholder='Select a Launch Site',
                                ),
                                            html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),

                                
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider( 
                                    id='payload-slider',
                                    min=min_payload,
                                    max=max_payload,
                                    step=1000,
                                    marks={min_payload: str(min_payload), max_payload: str(max_payload)},
                                    value=[min_payload, max_payload]
                                ),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])
                               




# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output('success-pie-chart', 'figure'), Input('site-dropdown', 'value'))
def update_pie_chart(selected_site):
            if selected_site == 'ALL':
                # Show total successful launches
                fig = px.pie(spacex_df, names='class', title='Success Count for All Sites')
                fig.update_traces(textinfo='label+percent', pull=[0, 0.1])
            else:
                # Filter data for the selected site and show Success vs. Failed counts
                site_data = spacex_df[spacex_df['Launch Site'] == selected_site]
                fig = px.pie(site_data, names='class', title=f'Success Count for {selected_site}')
                fig.update_traces(textinfo='label+percent', pull=[0, 0.1])

            return fig
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output('success-payload-scatter-chart', 'figure'), [Input('site-dropdown', 'value'), Input('payload-slider', 'value')])
def update_scatter_chart(selected_site, payload_range):
    min_payload, max_payload = payload_range
    if selected_site == 'ALL':
                                        # Show a scatter chart for all sites
        fig = px.scatter(spacex_df, x='Payload Mass (kg)', y='class', color='Booster Version Category',
                                                        title='Payload vs. Launch Success for All Sites')
    else:
                                        # Filter data for the selected site
        site_data = spacex_df[spacex_df['Launch Site'] == selected_site]
        fig = px.scatter(site_data, x='Payload Mass (kg)', y='class', color='Booster Version Category',
                                                        title=f'Payload vs. Launch Success for {selected_site}')
        fig.update_xaxes(range=[min_payload, max_payload])

    return fig

# Run the app
if __name__ == '__main__':
    app.run_server()
